In [1]:
from lib.util import *
from lib.policy import *
from lib.mdp import *
from lib.mrp import *

# Environment

In [2]:
from lib.env import *

# Tabular Monte Carlo

In [3]:
from lib.monte_carlo import *

$$V(S_t) \leftarrow V(S_t) + \alpha (G_t - V(S_t))$$

# Tabular Temporal Difference

In [4]:
from lib.temporal_difference import *

- Update value V(St) toward estimated return $R_{t+1} + γV(S_{t+1})$:
$$V(S_t) ← V(S_t) + α (R_{t+1} + γV(S_{t+1}) − V(S_t))$$
- $R_{t+1} + γV(S_{t+1})$ is called the TD target
- $δ_t = R_{t+1} + γV(S_{t+1}) − V(S_t)$ is called the TD error

# Example

###### MDP

In [7]:
n = 7
gamma = 0.8

In [8]:
P = generate_stochastic_matrix(n)
R = generate_reward_vector(n)
mrp = MRP(P, R, gamma)
mdp = MDP(gamma, [mrp]*n)
Q = generate_stochastic_matrix(n)
policy = Policy(Q)

###### Environment

In [9]:
env = Env(mdp)
print(env.reset())
print(env.step(0, 1))
print(env.step(2, 1))

5
(4, 0.8400953273403831, False, None)
(5, 0.2910977751764344, False, None)


###### Algorithm

In [10]:
env = Env(mdp)

In [11]:
monte_carlo(env, policy)

defaultdict(float,
            {4: 30.43642742984064,
             3: 30.66328143825837,
             0: 30.153235769665404,
             2: 29.94048229845399,
             1: 29.61606313921253,
             6: 30.224857761304513,
             5: 30.55238694969633})

In [12]:
td_zero(env, policy)

defaultdict(float,
            {0: 5.256349268833475,
             6: 4.911827447065534,
             5: 4.977053041564628,
             4: 4.785724497347818,
             3: 4.452810077337183,
             2: 4.906325733489601,
             1: 5.290377301951326})

In [13]:
forward_td_lambda(env, policy)

In [14]:
backward_td_lambda(env, policy)

### Proof that fixed learning rate $\alpha$ for MC is equivalent to an exponentially decaying average of episode returns

\begin{align*}
\mu_k &= \mu_{k-1} + \alpha(x_k - \mu_{k-1}) \\
&= \alpha x_k + (1-\alpha)\mu_{k-1} \\
&= \alpha x_k + (1-\alpha)(\alpha x_{k-1} + (1-\alpha)\mu_{k-2}) \\
&= \alpha x_k + (1-\alpha)\alpha x_{k-1} + (1-\alpha)^2\mu_{k-2} \\
&= \alpha x_k + (1-\alpha)\alpha x_{k-1} + (1-\alpha)^2(\alpha x_{k-2} + (1-\alpha)\mu_{k-3}) \\
&= \alpha x_k + (1-\alpha)\alpha x_{k-1} + (1-\alpha)^2\alpha x_{k-2} + (1-\alpha)^3\mu_{k-3} \\
&= \dots \\
&= (1-\alpha)^kx_0 + \alpha\sum_{i=1}^k (1-\alpha)^{k-i} x_i
\end{align*}

### Proof that Offline Forward-View TD($\lambda$) and Offline Backward View TD($\lambda$) are equivalent

$E_t(s) = \gamma \lambda E_{t-1}(s) + \mathbb{1}(S_t = s) = (\gamma \lambda)^{t-k}\mathbb{1}(t \geq k)$

Backward $TD(\lambda)$ updates accumulate error online $\sum_{t=1}^T \alpha \delta_t E_t(s) = \alpha\sum_{t=k}^T(\gamma \lambda)^{t-k}\delta_t $

Moreover,
\begin{align*}
G_t^\lambda - V(S_t) 
&= -V(S_t) + (1-\lambda)\lambda^0 (R_{t+1}+\gamma V(S_{t+1})) + (1-\lambda)\lambda^1 (R_{t+1}+\gamma R_{t+2} +\gamma^2 V(S_{t+2})) + \dots \\
&= -V(S_t) + (\gamma \lambda)^0 (R_{t+1}+\gamma V(S_{t+1}) - \gamma\lambda V(S_{t+1})) + (\gamma \lambda)^1 (R_{t+2}+\gamma V(S_{t+2}) - \gamma\lambda V(S_{t+2})) + \dots \\
&= (\gamma \lambda)^0 (R_{t+1}+\gamma V(S_{t+1}) - V(S_{t})) + (\gamma \lambda)^1 (R_{t+2}+\gamma V(S_{t+3}) - V(S_{t+2})) + \dots \\
&= \sum_{t=k}^T(\gamma \lambda)^{t-k}\delta_t
\end{align*}

Hence, we get  $\sum_{t=1}^T \alpha \delta_t E_t(s) =  \sum_{t=1}^T \alpha (G_t^\lambda - V(S_t))$